In [3]:
%pylab inline
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Table

import numpy as np

from photutils import aperture_photometry, CircularAperture, SkyCircularAperture

Populating the interactive namespace from numpy and matplotlib


In [40]:
#coordinates
from astropy.io import fits
hdulist = fits.open('SNhunt175_r.fits')
#print (hdulist.info())
#print (hdulist[0].header)
RA = hdulist[0].header["RA"]
Dec = hdulist[0].header["DEC"]
print(RA)
print (Dec)

coordinates = SkyCoord(RA, Dec, unit = (u.hourangle, u.deg))
coordinates

229.00858
20.526425


<SkyCoord (ICRS): (ra, dec) in deg
    ( 195.1287,  20.526425)>

In [41]:
# Find the aperture radius in armin
t = Table.read("observed_target_info.dr1.txt", format = "ascii")
z = t["z"]
radius = 1 * u.kpc / cosmo.kpc_comoving_per_arcmin(.0409)
r = radius * 60 * u.arcsec / u.arcmin
r
#print(r)

#usually do concentric circles of aperture radius to figure out which size is the best to use
#assume right now 1 kpc

<Quantity 1.1765505181148288 arcsec>

In [44]:
# Create an aperture
aperture = SkyCircularAperture(coordinates, r)
aperture

In [72]:
# Perform photometryb
#with closes file when finished running code
with fits.open('SNhunt175_r.fits') as ofile:
    exp_time = ofile[0].header["EXPTIME"]
    data= ofile[0].data
    pixel_error = np.sqrt(ofile[0].data / float(exp_time))
    int_phot_table = aperture_photometry(ofile[0], aperture, error = pixel_error)

print(any(np.isnan(data)))

int_phot_table


False


C:\Users\Emily\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in sqrt
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


id,xcenter,ycenter,celestial_center,aperture_sum,aperture_sum_err
,pix,pix,,,
int32,float64,float64,object,float64,float64
1,-74416.93975366684,-320329.0461587422,"<SkyCoord (ICRS): (ra, dec) in deg ( 195.1287, 20.526425)>",nan,nan


In [ ]:
#want surface brightness (which I get by doing aperature photometry)
#separate image for each filter

#calibration is nanomaggies
    #one nanomaggie is something that's a brightness of 22.5 magnitudes
#log scale assumes positive flux which will confuse uncertaintiy esitmates at low detected photon values

#for test case, make sure that brightness I get is similar to brightness SDSS gets
#debugging test case, check what the simplest thing is that makes it not work

In [58]:
#learning basic photometry
from photutils import CircularAperture
positions = [(30., 30.), (40., 40.)]
apertures = CircularAperture(positions, r = 3.)
data = np.ones([100, 100])
error = 0.1 * data
phot_table = aperture_photometry(data, apertures, error=error)
phot_table

id,xcenter,ycenter,aperture_sum,aperture_sum_err
,pix,pix,,
int32,float64,float64,float64,float64
1,30.0,30.0,28.2743338823,0.531736155272
2,40.0,40.0,28.2743338823,0.531736155272


In [65]:
#test case with negative numbers
from photutils import CircularAperture
positions = [(3., 3.), (4., 4.)]
apertures = CircularAperture(positions, r = 3.)
data = np.array([[1, 1,1, 1], [1, 1, 1, 1], [1, -1, 1, 1], [1, 1, 1, 1]])
phot_table = aperture_photometry(data, apertures)
phot_table

id,xcenter,ycenter,aperture_sum
,pix,pix,
int32,float64,float64,float64
1,3.0,3.0,8.30463612933
2,4.0,4.0,4.31664520607


In [36]:
#test cast skycircular coordinates and uploaded data
from astropy.coordinates import SkyCoord
from photutils import SkyCircularAperture
from astropy import wcs
from photutils import datasets
hdu = datasets.load_spitzer_image()
catalog = datasets.load_spitzer_catalog()
#positions = SkyCoord(catalog['l'], catalog['b'], unit = (u.arcsec, u.arcsec), frame = 'galactic')
#apertures = SkyCircularAperture(positions, r = 4.8 *u.arcsec)
phot_table = aperture_photometry(hdu, aperture)
phot_table

id,xcenter,ycenter,celestial_center,aperture_sum
,pix,pix,"deg,deg",
int32,float64,float64,object,float64
1,-250789.04905435123,179140.50786607957,"210.77416666666667,54.273694444444445",nan
2,-492859.9802108787,119446.01986013724,"133.69189999999998,39.53778333333333",nan
3,-402362.0589039787,-38638.22213411843,"52.39716249999999,40.86748055555556",nan
4,-461592.4336746684,-182942.33264130194,"34.690837499999994,-6.641791666666666",nan
5,-217369.6798050777,-52931.27702441495,"334.3312875,35.580538888888896",nan
6,-73014.98062331845,-104802.80326585437,"318.1012583333333,-7.814455555555555",nan
7,-216700.94837411016,-146736.7247911804,"351.9812833333333,8.779291666666667",nan
8,-500003.945008993,222137.56614267643,"176.05441666666664,33.51663611111111",nan


In [ ]:
#want to figure out why NaN return